<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 4 Assignment: Classification and Regression Neural Network**

**Student Name: Hao-Tien Kuo**

# Assignment Instructions

For this assignment, you will use the **crx.csv** dataset.  This dataset is a public dataset that can you can find [here](https://archive.ics.uci.edu/ml/datasets/credit+approval). You should use the CSV file on my data site, at this location: [crx.csv](https://data.heatonresearch.com/data/t81-558/crx.csv) because it includes column headers.  The primary use for this dataset is binary classification. There are 15 attributes, plus a target column that contains only + or -.  Some of the columns have missing values.

You should train a neural network and return the predictions.  You will submit these predictions to the **submit** function.  See [Assignment #1](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class1.ipynb) for details on how to submit an assignment or check that one was submitted.

Complete the following tasks:

* Your task is to replace missing values in columns *a2* and *a14* with values estimated by a neural network (one neural network for *a2* and another for *a14*).
* Your submission file will contain the same headers as the source CSV: *a1*, *a2*, *s3*, *a4*, *a5*, *a6*, *a7*, *a8*, *a9*, *a10*, *a11*, *a12*, *a13*, *a14*, *a15*, and *a16*.
* You should only need to modify *a2* and *a14*.
* Neural networks can be much more powerful at filling missing variables than median and mean.
* Train two neural networks to predict *a2* and *a14*.  
* The *y* (target) for training the two nets will be *a2* and *a14*, depending on which you are trying to fill.
* The x for training the two nets will be 's3','a8','a9','a10','a11','a12','a13','a15'.  These are chosen because it is important not to use any columns with missing values; also, it could cause unwanted bias if we include the ultimate target (*a16*).
* ONLY predict new values for missing values in *a2* and *a14*.
* You will likely get this small warning:  Warning: The mean of column a14 differs from the solution file by 0.20238937709643778. (might not matter if small)



# Google CoLab Instructions

If you are using Google CoLab, it will be necessary to mount your GDrive so that you can send your notebook during the submit process. Running the following code will map your GDrive to ```/content/drive```.

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Assignment Submit Function

You will submit the ten programming assignments electronically.  The following **submit** function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any underlying problems. 

**It is unlikely that should need to modify this function.**

In [2]:
import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    payload = []
    for item in data:
        if type(item) is PIL.Image.Image:
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG':base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif type(item) is pd.core.frame.DataFrame:
            payload.append({'CSV':base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
    r= requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={ 'payload': payload,'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code==200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Assignment #4 Sample Code

The following code provides a starting point for this assignment.

In [254]:
import os
import pandas as pd
from scipy.stats import zscore
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics

# This is your student key that I emailed to you at the beginnning of the semester.
key = "lhkxWaPDTw1k2isuaX0Gk4X0XN0RcGFX3lEQFj0l"  # This is an example key and will not work.

# You must also identify your source file.  (modify for your local setup)
file='/content/drive/MyDrive/Colab Notebooks/Deep Neural Networks/assignment_Hao-Tien Kuo_class4.ipynb'  # Google CoLab

# Begin assignment
df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/crx.csv",na_values=['?'])

In [256]:
# Below is just a suggestion.  These are the imports that I used.
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

def fill_missing_numeric(df,current,target):
  # Fill in as needed
  df_original = df.copy()

  # Generate dummies 
  df = pd.concat([df,pd.get_dummies(df['a9'],prefix="a9")],axis=1)
  df = pd.concat([df,pd.get_dummies(df['a10'],prefix="a10")],axis=1)
  df = pd.concat([df,pd.get_dummies(df['a12'],prefix="a12")],axis=1)
  df = pd.concat([df,pd.get_dummies(df['a13'],prefix="a13")],axis=1)
  df.drop(['a9','a10','a12','a13'], axis=1, inplace=True)

  # Standardize ranges
  df['s3'] = zscore(df['s3'])
  df['a8'] = zscore(df['a8'])
  df['a11'] = zscore(df['a11'])
  df['a15'] = zscore(df['a15'])

  # Convert to numpy
  x_columns = df.columns.drop('a1').drop('a2').drop('a4').drop('a5').drop('a6').drop('a7').drop('a14').drop(target)
  x = df.dropna(subset=['a2', 'a14'])[x_columns].values
  y = df.dropna(subset=['a2', 'a14'])[current].values

  # Create train/validation
  x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=10)

  # Build the neural network
  model = Sequential()
  model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1
  model.add(Dense(10, activation='relu')) # Hidden 2
  model.add(Dense(1)) # Output
  model.compile(loss='mean_squared_error', optimizer='adam')
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
                          patience=5, verbose=0, mode='auto', 
                          restore_best_weights=True)
  model.fit(x_train,y_train,validation_data=(x_val,y_val),
            callbacks=[monitor],verbose=0,epochs=1000)
  
  # Fill na with predictions
  for i in df[df[current].isna()].index:
    df_original[current][i] = model.predict(df.loc[[i], x_columns].values)[0][0]

  return df_original

df_submit = fill_missing_numeric(df,'a2','a16')
df_submit = fill_missing_numeric(df_submit,'a14','a16')
# Submit

submit(source_file=file,data=[df_submit],key=key,no=4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Success: Submitted Assignment 4 for h.kuo:
You have submitted this assignment 4 times. (this is fine)
Note: The mean difference 0.003817993446848078 for column 'a2' is acceptable and is less than the maximum allowed value of '1.0' for this assignment.
Note: The mean difference 0.2998341325400702 for column 'a14' is acceptable and is less than the maximum allowed value of '1.0' for this assignment.
No warnings or errors (only notes), you will probably do well, but no guarantee. :-)
